# SQL Snippets (WIP)

## Fetching historical data for specific dates

Some tables in the warehouse--like those in `gtfs_schedule_type2`--capture the full
history of data, as it existed every day in the past. In order to do this, they
don't save copies of the full data every day, but log changes to the data using
a `calitp_extracted_at`, and `calitp_deleted_at` column.

In order to get the data for a given day, you need to filter to keep data where..

* `calitp_extracted_at` was earlier than or on the target date.
* `calitp_deleted_at` is later than the target date.

## A single date

In [1]:
from calitp.tables import tbl
from myst_nb import glue
from calitp import query_sql
from siuba import *
import pandas as pd
import calitp.magics

In [2]:
%%sql -m
SELECT *
FROM `gtfs_schedule_type2.feed_info`
WHERE
    calitp_extracted_at >= "2021-06-01"
    AND COALESCE(calitp_deleted_at, "2099-01-01") > "2021-06-01"

```SQL
SELECT *
FROM `gtfs_schedule_type2.feed_info`
WHERE
    calitp_extracted_at >= "2021-06-01"
    AND COALESCE(calitp_deleted_at, "2099-01-01") > "2021-06-01"

```

,calitp_itp_id,calitp_url_number,feed_publisher_name,feed_publisher_url,feed_lang,default_lang,feed_start_date,feed_end_date,feed_version,feed_contact_email,feed_contact_url,calitp_extracted_at,calitp_deleted_at,calitp_hash
0,264,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20210623,20220701,UTC: 23-Jun-2021 22:39,support+test+riovista-ca-us@trilliumtransit.com,http://support.trilliumtransit.com,2021-06-23,2021-06-24,mcskNy04Xp6Ck8+XM2/t0Q==
1,217,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20210623,20220101,UTC: 23-Jun-2021 20:45,support+test+mountainview-ca-us@trilliumtransi...,http://support.trilliumtransit.com,2021-06-23,2021-06-24,A9R3MhcDkpJVkswQx8B0SA==
2,135,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20210615,20220101,UTC: 15-Jun-2021 21:23,support+test+humboldtcounty-ca-us@trilliumtran...,http://support.trilliumtransit.com,2021-06-15,2021-06-24,f9hhEXyAz6Xzw2BEa+lIKg==
3,42,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20210615,20220101,UTC: 15-Jun-2021 21:23,support+test+humboldtcounty-ca-us@trilliumtran...,http://support.trilliumtransit.com,2021-06-15,2021-06-24,wr5pJIbn3i6IWzVV0lY+WA==
4,18,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20210615,20220101,UTC: 15-Jun-2021 21:23,support+test+humboldtcounty-ca-us@trilliumtran...,http://support.trilliumtransit.com,2021-06-15,2021-06-24,H3xOwcVCzviy0e/2fBOW7A==
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654,110,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20210628,20211231,UTC: 28-Jun-2021 18:48,support+test+fairfield-ca-us@trilliumtransit.com,http://support.trilliumtransit.com,2021-06-28,2021-07-09,dv+EUFVzgrGiPRmqyukgpw==
655,101,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20210625,20220101,UTC: 25-Jun-2021 23:49,support+test+eldoradotransit-ca-us@trilliumtra...,http://support.trilliumtransit.com,2021-06-26,2021-07-09,FmB76frnyFcZzonHQSzyEw==
656,280,1,511 SF Bay,https://511.org/open-data/transit,en,None,20210701,20220101,20210705,None,None,2021-07-05,2021-07-16,Y9HGQnDB03hckt7vkF8IRQ==
657,280,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20210705,20220101,UTC: 05-Jul-2021 19:37,support+test+sfbayferry-ca-us@trilliumtransit.com,http://support.trilliumtransit.com,2021-07-05,2021-07-16,2XPu8wINklW3O3UqaaaFSA==


Note that `COALESCE` lets us fill in NULL deleted at values to be far in the future.
This is used because when deleted at is missing, it reflects the most recent data
(i.e. data that hasn't been deleted yet).
Because `NULL < "2021-06-01"` is `false`, we need to fill it in with a far-future date,
so it evaluates to `true`.

## Multiple dates

In order to do it for a range of dates, you can use a JOIN. This is shown below.

In [3]:
%%sql -m
SELECT *
FROM `gtfs_schedule_type2.feed_info` FI
JOIN `views.dim_date` D
    ON FI.calitp_extracted_at <= D.full_date
        AND COALESCE(FI.calitp_deleted_at, "2099-01-01") > D.full_date
WHERE
    D.full_date BETWEEN "2021-06-01" AND "2021-06-07"

```SQL
SELECT *
FROM `gtfs_schedule_type2.feed_info` FI
JOIN `views.dim_date` D
    ON FI.calitp_extracted_at <= D.full_date
        AND COALESCE(FI.calitp_deleted_at, "2099-01-01") > D.full_date
WHERE
    D.full_date BETWEEN "2021-06-01" AND "2021-06-07"

```

,calitp_itp_id,calitp_url_number,feed_publisher_name,feed_publisher_url,feed_lang,default_lang,feed_start_date,feed_end_date,feed_version,feed_contact_email,...,month_name,week_day,day_name,day_is_weekday,is_quarter_start,is_month_start,is_in_past,is_in_past_or_present,is_in_future,is_gtfs_schedule_range
0,226,0,NCTD,http://www.gonctd.com,EN,None,None,None,Version_2021-05-14,None,...,June,3,Wednesday,1,False,False,True,True,False,True
1,226,0,NCTD,http://www.gonctd.com,EN,None,None,None,Version_2021-05-14,None,...,June,1,Monday,1,False,False,True,True,False,True
2,226,0,NCTD,http://www.gonctd.com,EN,None,None,None,Version_2021-05-14,None,...,June,2,Tuesday,1,False,True,True,True,False,True
3,226,0,NCTD,http://www.gonctd.com,EN,None,None,None,Version_2021-05-14,None,...,June,6,Saturday,0,False,False,True,True,False,True
4,226,0,NCTD,http://www.gonctd.com,EN,None,None,None,Version_2021-05-14,None,...,June,0,Sunday,0,False,False,True,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,389,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20210406,20210630,UTC: 07-Apr-2021 00:54,support+test+nlte-nv-us@trilliumtransit.com,...,June,2,Tuesday,1,False,True,True,True,False,True
1305,389,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20210406,20210630,UTC: 07-Apr-2021 00:54,support+test+nlte-nv-us@trilliumtransit.com,...,June,6,Saturday,0,False,False,True,True,False,True
1306,389,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20210406,20210630,UTC: 07-Apr-2021 00:54,support+test+nlte-nv-us@trilliumtransit.com,...,June,0,Sunday,0,False,False,True,True,False,True
1307,389,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20210406,20210630,UTC: 07-Apr-2021 00:54,support+test+nlte-nv-us@trilliumtransit.com,...,June,4,Thursday,1,False,False,True,True,False,True
